* The objective of this assignments is to build the **Decoder** part of the Transformer architecture.
* We will be using the **PyTorch** framework to implement the following components
  * Decoder Layer that contains
    * Multi-Head Masked Attention (MHMA) Module
    * Multi-Head Cross Attention (MHMA) Module
    * Position-wise Feed Forward Neural Network

  * Implement CLM

* **DO NOT** USE Built-in **TRANSFORMER LAYERS** as it affects the reproducibility.

* You will be given with a configuration file that contains information on various hyperparameters such as embedding dimension, vocabulary size,number heads and so on

* Use ReLU activation function and Stochastic Gradient Descent optimizer
* Here are a list of helpful Pytorch functions (does not mean you have to use all of them) for this subsequent assignments
  * [torch.matmul](https://pytorch.org/docs/stable/generated/torch.matmul.html#torch-matmul)
  * [torch.bmm](https://pytorch.org/docs/stable/generated/torch.bmm.html)
  * torch.swapdims
  * torch.unsqueeze
  * torch.squeeze
  * torch.argmax
  * [torch.Tensor.view](https://pytorch.org/docs/stable/generated/torch.Tensor.view.html)
  * [torch.nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)
  * [torch.nn.Parameter](https://pytorch.org/docs/stable/generated/torch.nn.parameter.Parameter.html)
  * torch.nn.Linear
  * torch.nn.LayerNorm
  * torch.nn.ModuleList
  * torch.nn.Sequential
  * [torch.nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)
  
* Important: Do not set any global seeds.

* Helpful resources to get started with

 * [Andrej Karpathys Nano GPT](https://github.com/karpathy/nanoGPT)
 * [PyTorch Source code of Transformer Layer](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html)



In [ ]:
import torch
from torch import Tensor

import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.nn.functional import one_hot

import torch.optim as optim

from  pprint import pprint
from yaml import safe_load
import copy
import requests
from io import BytesIO

In [ ]:
#do not edit this cell
config_url = "https://raw.githubusercontent.com/Arunprakash-A/LLM-from-scratch-PyTorch/main/config_files/dec_config.yml"
response = requests.get(config_url)
config = response.content.decode("utf-8")
config = safe_load(config)
pprint(config)

In [ ]:
vocab_size = config['input']['vocab_size']
batch_size = config['input']['batch_size']
seq_len = config['input']['seq_len']
embed_dim = config['input']['embed_dim']
dmodel = embed_dim
dq = torch.tensor(config['model']['dq'])
dk = torch.tensor(config['model']['dk'])
dv = torch.tensor(config['model']['dv'])
heads = torch.tensor(config['model']['n_heads'])
d_ff = config['model']['d_ff']

# Input tokens

* Generate a raw_input ids (without any special tokens appended to it)

* Since we will be using this as label after adding the special  \<start\> token, we use the variable name "label_ids"

* Keep the size of the `label_ids=(bs,seq_len-1)` as we insert a special token ids in the next step

In [ ]:
# do not edit this cell
data_url = 'https://github.com/Arunprakash-A/LLM-from-scratch-PyTorch/raw/main/config_files/w2_input_tokens'
r = requests.get(data_url)
label_ids = torch.load(BytesIO(r.content))

* Let the first token_id be be a special `[start]` token (mapped to integer 0)
* If label_ids=$\begin{bmatrix}1&2\\3&4 \end{bmatrix}$, then we modify it as $\begin{bmatrix}0&1&2\\0&3&4 \end{bmatrix}$

In [ ]:
token_ids = None # the first column of token_ids should be zeros and the rest of the columns come from label_ids

# Implement the following components of a decoder layer

 * Multi-head Masked Attention (MHMA)
 * Multi-head Cross Attention (MHCA)
 * Postion-wise FFN

* Randomly initialize the parameters using normal distribution with the following seed values
  * $W_Q:$(seed=43)
  * $W_K:$(seed=44)
  * $W_V:$(seed=45)
  * $W_O:$(seed=46)

* Remember that, Multi-head cross atention takes two represnetation. One is the encoder output and the other one is the output from masked attetnion sub-layer.

* However, in this assignment, we will fix it to a random matrix.

In [ ]:
class MHCA(nn.Module):

  def __init__(self,dmodel,dq,dk,dv,heads):
    super(MHCA,self).__init__()


     # your method definitions go here (if you want to)

  def forward(self):


    return out


* By default, `mask=None`. Therefore, create and apply the mask while computing the attention scores


In [ ]:
class MHMA(nn.Module):

  def __init__(self,dmodel,dq,dk,dv,heads,mask=None):
    super(MHMA,self).__init__()
    # your code goes here



  def forward(self):
    # implement forward method
    return out

* Implement the FFN and OutputLayer modules (same as the one you implemented for encoder)

In [ ]:
class FFN(nn.Module):
  def __init__(self,dmodel,d_ff):
    super(FFN,self).__init__()


  def forward(self,x):

    return out

In [ ]:
class OutputLayer(nn.Module):

  def __init__(self,dmodel,vocab_size):
    super(OutputLayer,self).__init__()


  def forward(self,):

    return out

* Implement the final decoder layer.

In [ ]:
class DecoderLayer(nn.Module):

  def __init__(self,dmodel,dq,dk,dv,d_ff,heads,mask=None):
    super(DecoderLayer,self).__init__()
    self.mhma = MHMA(dmodel,dq,dk,dv,heads,mask=None)
    self.mhca = MHCA(dmodel,dq,dk,dv,heads)
    self.layer_norm_mhma = torch.nn.LayerNorm(dmodel)
    self.layer_norm_mhca = torch.nn.LayerNorm(dmodel)
    self.layer_norm_ffn = torch.nn.LayerNorm(dmodel)
    self.ffn = FFN(dmodel,d_ff)

  def forward(self,):

    return out

* Create an embedding layer that takes in token_ids and return embeddings for the token_ids

 * Use seed value: 70

In [ ]:
class Embed(nn.Module):

  def __init__(self,vocab_size,embed_dim):
    super(Embed,self).__init__()

    self.embed= None

  def forward(self,x):
    out = self.embed(x)
    return out

# Decoder

 * Implement the decoder that has `num_layers` decoder layers

In [ ]:
class Decoder(nn.Module):

  def __init__(self,vocab_size,dmodel,dq,dk,dv,d_ff,heads,mask,num_layers=1):
    super(Decoder,self).__init__()
    self.embed_lookup = None
    self.dec_layers = None
    self.out = None

  def forward(self,enc_rep,tar_token_ids):


    return out=None

* Representation from encoder

 * Since all the decoder layers require the representation from the encoder to compute cross attention, we are going to feed in the random values (Note, it does not require gradient during training)

In [ ]:
# do not edit this
enc_rep = torch.randn(size=(batch_size,seq_len,embed_dim),generator=torch.random.manual_seed(10))

# Instantiate the model

In [ ]:
model = Decoder(vocab_size,dmodel,dq,dk,dv,d_ff,heads,mask=None)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
def train(enc_rep,tar_token_ids,label_ids,epochs=1000):
  loss_trace = []
  for epoch in range(epochs):
    out = model(enc_rep,tar_token_ids)
    loss =
    loss.backward()

    #update parameters
    optimizer.step()
    optimizer.zero_grad()

* Train the model for 1000 epochs

In [ ]:
train(enc_rep,token_ids,label_ids,1000)

In [ ]:
with torch.inference_mode():
  predictions = torch.argmax(model(enc_rep,token_ids),dim=-1)

* The loss will be around 0.17 after 1000 epochs

In [ ]:
# number of correct predictions
print(torch.count_nonzero(label_ids==predictions[:,0:-1]))

* THe number of correct predictions is close to 66